### Branching and Tagging - Visão Geral

As tabelas do Iceberg matem um log de snapshots, que por sua vez representão as mudanças aplicadas na tabela.
Os snapshots são fundamentais no formato de tabela Iceberg, pois eles garantem o isolamento das onsultas/queries e permitem o "time travel" entre as diferentes "versões de dados" da tabela.

Para carontrolar o tamanho dos metadados e custos de armazenamento, Iceberg fornece um gerenciamento de ciclo de vida de snapshots. Que pode ser executado via procedures como **_expire_snapshots_** para remoção de de arquivos de dados não utilizados com base nas propriedades dos snapshots.

Para possibilidar o gerenciamento de ciclo de vidade de snapshots. O Iceberg suporta Branches e Tags, Esses recursos garantem que os snapshots tenham seus proprios ciclos de vida que por sua vez podem ser controlados por politicas de retenção em nivel de Branch e tag.

**Branches (Ramificações):** São referências mutáveis que podem ser atualizadas para apontar para um novo snapshot. Isso significa que uma branch pode evoluir ao longo do tempo, acompanhando as mudanças na tabela. Uma branch é como uma linha de desenvolvimento independente, permitindo que você faça alterações sem afetar a "linha principal" de desenvolvimento (main branch).

**Tags (Etiquetas):** São referências imutáveis a um snapshot específico. Uma vez criada, uma tag sempre apontará para o mesmo snapshot, independentemente de quaisquer alterações subsequentes na tabela. As tags servem como marcadores fixos no tempo, permitindo que você retorne a um estado específico da tabela com facilidade.

fonte: https://iceberg.apache.org/docs/1.6.1/branching/

### Write-Audit-Publish (WAP)
Branch no Iceberg é similar ao sistema de controle de versionamento de código git.

Com as Branches você poderar perfeitamente controlar e gerenciar as versões dos seus dados de forma isolada, assim como ocorre na Engenharia de Software com os códigos. A partir de agora, no Apache Iceberg podemos falar do padrão de engenharia de dados WAP que visa garantir a qualidade dos dados antes de desponibiliza-los para consumo.<br>

**Write (Escrita):** Os dados são gravados inicialmente em um local de teste ou staging, em vez de serem gravados diretamente na tabela principal. Isso isola os dados de produção de possíveis inconsistências. No Apache Iceberg, isso geralmente é feito gravando em um branch separado da tabela principal.

**Audit (Auditoria):** Uma vez que os dados estão em staging, eles são submetidos a um processo de validação completo para garantir que atendam aos padrões de qualidade. Isso pode incluir a verificação de valores nulos ou duplicados, validação de tipos de dados e verificação da integridade dos dados. Qualquer ferramenta de qualidade de dados pode ser usada para essa etapa.

**Publish (Publicação):** Após a validação, os dados são promovidos para a tabela de produção. No Iceberg, isso geralmente é feito por meio de um fast-forward do branch de staging para o branch principal. 

![ci/cd](https://www.tabular.io/wp-content/uploads/2024/01/image1-1024x182.png) <br>

fonte: https://www.tabular.io/apache-iceberg-cookbook/data-engineering-write-audit-publish/

### Carregar Recursos

In [ ]:
# Importar funções 
from IPython.display import display, HTML

In [ ]:
%run ./Includes/Utils.ipynb

In [ ]:
## Listar branches da tabela

spark.sql("SELECT * FROM iceberg.bronze.vendas.refs;").toPandas()

### Ativar o Write-Audit-Publish(WAP) e Criar a nova Branche

In [ ]:
## Ativar WAP
spark.sql("""
    ALTER TABLE iceberg.bronze.vendas
    SET TBLPROPERTIES (
        'write.wap.enabled' = 'true'
    )"""
)

In [ ]:
# Criar uma branch chamada dev_etl

spark.sql("ALTER TABLE iceberg.bronze.vendas CREATE BRANCH dev_etl").show()

### <span style="color:blue"> Quando for trabalhar com as branches, é importante adicionar o prefixo``` branch_ ``` ao nome da nova branch </span>.

- <span style="color:red">Cuidado para não inserir dados por engano na branch ``` main ``` </span>.
- <span style="color:red">Caso isso ocorra após a criação da branch de etl, você perderar a referencia da branch **'pai'** e não conseguirá fazer um merge </span>.
- <span style="color:red">Caso você perca a referencia do ultimo estado 'válido/correto' da tabela. Terá que fazer um rollback para o ultimo snapshot sem alterações </span>.


### Inspecionar tabela

In [ ]:
## Exibir os dados da brach dev_etl ===> Quando for listar uma branche é importante adicionar o prefixo branch_dev_etl

spark.sql("SELECT * FROM iceberg.bronze.vendas.branch_dev_etl").toPandas()

In [ ]:
## Listar branches

spark.sql("SELECT * FROM iceberg.bronze.vendas.refs;").toPandas()

In [ ]:
## Listar snapshots

spark.sql("SELECT * FROM iceberg.bronze.vendas.snapshots;").toPandas()

In [ ]:
## Listar tabela por versão main
spark.sql("SELECT * FROM iceberg.bronze.vendas VERSION AS OF 'main' ")

In [ ]:
## Listar tabela por versão dev_etl
spark.sql("SELECT * FROM iceberg.bronze.vendas VERSION AS OF 'dev_etl' ")

### Inserir Novos dados na branch dev

In [ ]:
## Inserir novos dados

mar_data_df = create_dataframe(columns_schema, mar_data)

mar_data_df.writeTo("iceberg.bronze.vendas.branch_dev_etl").append()

In [ ]:
## Inserir novos dados

# apr_data_df = create_dataframe(columns_schema, apr_data)

# apr_data_df.writeTo("iceberg.bronze.vendas.branch_dev_etl").append()

In [ ]:
## Exibir novamente os dados da brach dev_branch

spark.sql("SELECT * FROM iceberg.bronze.vendas.branch_dev_etl").toPandas()

In [ ]:
## Exibir os dados da brach main

spark.sql("SELECT * FROM iceberg.bronze.vendas").toPandas()

### Fast forward - Merge da dev_branch na branch main
Para publicar a branch ``` 'dev' ``` na branch ``` 'main' ``` basta usar a procedure: catalog.system.fast_forward('catalog.db.tabela', 'main', 'dev').<br>
 
Esta procedure faz uma mescla entre as duas versões da tabela, o processo é similar a mesclar um pull request no Git.
 
Esta procedure moverá o ponteiro do branch main para o mesmo commit do branch ``` 'dev' ``` , e efetivamente faz a publicação das alterações na branch principal (``` 'main' ```).

In [ ]:
spark.sql(f" CALL iceberg.system.fast_forward('iceberg.bronze.vendas', 'main', 'dev_etl')")

### Se tudo deu certo vamos deletar a branch de dev

In [2]:
## Em caso de sucesso ou falha é bom não deixar historico desnecessário ==> Boas praticas do padrão Feature-Branch
spark.sql("ALTER TABLE iceberg.bronze.vendas DROP BRANCH dev_etl")

In [ ]:

#Exibir versão final
spark.sql("SELECT * FROM iceberg.bronze.vendas").toPandas()